In [28]:
import numpy as np
import pyedflib
import statistics
import plotly.graph_objects as go
import pandas as pd
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy, Amplitude, NumberOfPoints, ComplexPolynomial, PersistenceLandscape, HeatKernel, Silhouette, BettiCurve, PairwiseDistance, ForgetDimension
from gtda.plotting import plot_point_cloud, plot_heatmap, plot_diagram
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA, FastICA
from gtda.pipeline import Pipeline 

In [15]:
label_list = [0, 1, 2, 3, 4]

In [16]:
# Load persistence diagrams

train_persistence_diagrams = {} # dictionary with labels as keys, persistence diagrams of the respective classes as values
test_persistence_diagrams = {} # dictionary with labels as keys, persistence diagrams of the respective classes as values


for label in label_list:
    train_persistence_diagrams[label] = np.load("Embeddings_and_Persistence_Diagrams/Train_PD"+str(label)+".npy", allow_pickle=True)
    test_persistence_diagrams[label] = np.load("Embeddings_and_Persistence_Diagrams/Test_PD"+str(label)+".npy", allow_pickle=True)


# Summary Statistics

In [19]:
def compute_summary_statistics(persistence_diagrams):
    """
    Compute summary statistics of list of persistence diagrams

    Parameters:
    - persistence_diagrams (list): persistence diagrams

    Returns:
    Tuple of four lists:
    - Persistence Entropy
    - Persistence
    - Betti Numbers
    - Complex Polynomials
    """
    
    PE = PersistenceEntropy()
    AM = Amplitude()
    NP = NumberOfPoints()
    CP = ComplexPolynomial(n_coefficients=1)

    persistence_entropies = []
    amplitudes = []
    nos_points = []
    complex_polynomials = []

    for diagram in persistence_diagrams:
        persistence_entropies.append(PE.fit_transform([diagram]))
        amplitudes.append(AM.fit_transform([diagram]))
        nos_points.append(NP.fit_transform([diagram]))
        #complex_polynomials.append(CP.fit_transform([diagram]))

    return persistence_entropies, amplitudes, nos_points, #complex_polynomials

In [20]:
train_feautures = {}
test_feautures = {}

for label in label_list:
    train_feautures[label] = compute_summary_statistics(train_persistence_diagrams[label])
    test_feautures[label] = compute_summary_statistics(test_persistence_diagrams[label])

In [21]:
def compute_largest_persistence(persistence_diagrams):
    """
    Computes persistence of the most prominent points of each dimension in each diagram

    Parameters:
    - persistence_diagrams (list): persistence diagrams

    Returns:
    List of 3 lists:
    - List of the largest persistences of homology dimension 0
    - List of the largest persistences of homology dimension 1
    - List of the largest persistences of homology dimension 2 
    """

    largest_persistences = [] # will contain 3 lists for the 3 homology dimensions
    for homology_dimension in [0, 1, 2]:
        largest_persistences_of_hom_dim = []
        for diagram in persistence_diagrams:
            # only look at holes of our homology dimension
            condition = diagram[:, 2] == homology_dimension
            filtered_diagram = diagram[condition]

            if len(filtered_diagram) > 0:
                differences = filtered_diagram[:, 1] - filtered_diagram[:, 0]
                largest_persistences_of_hom_dim.append(np.max(differences))

        largest_persistences.append(largest_persistences_of_hom_dim)

    return largest_persistences
    

    

In [22]:
train_largest_persistences = {}
test_largest_persistences= {}

for label in label_list:
    train_largest_persistences[label] = compute_largest_persistence(train_persistence_diagrams[label])
    test_largest_persistences[label] = compute_largest_persistence(test_persistence_diagrams[label])

# Concatenate Features to one DataFrame

In [23]:
def choose_column_in_matrix(matrix, i):
    return [row[0][i] for row in matrix]

In [24]:
def create_feature_df(persistence_entropies, amplitudes, nos_points, persistences, label):
    """
    Create DataFrame for each label from features

    Parameters:
    - persistence_entropies (list): persistence entropies
    - amplitudes (list): amplitudes
    - nos_points (list): number of points
    - label (int): Label for which we want to create a dataframe. 1, 3, 5 or 7.

    Returns:
    - Feature DataFrame (DataFrame)
    """
    
    feature_df = pd.DataFrame()

    # All 3 columns (corresponding to hole dimensions)
    feature_df["Persistence Entropy_Dim_0"] = choose_column_in_matrix(list(persistence_entropies), 0)
    feature_df["Persistence Entropy_Dim_1"] = choose_column_in_matrix(list(persistence_entropies), 1)
    feature_df["Persistence Entropy_Dim_2"] = choose_column_in_matrix(list(persistence_entropies), 2)

    # All 3 columns (corresponding to hole dimensions)
    feature_df["Amplitude_Dim_0"] = choose_column_in_matrix(list(amplitudes), 0)
    feature_df["Amplitude_Dim_1"] = choose_column_in_matrix(list(amplitudes), 1)
    feature_df["Amplitude_Dim_2"] = choose_column_in_matrix(list(amplitudes), 2)

    # All 3 columns (corresponding to hole dimensions)
    feature_df["No_Points_Dim_0"] = choose_column_in_matrix(list(nos_points), 0)
    feature_df["No_Points_Dim_1"] = choose_column_in_matrix(list(nos_points), 1)
    feature_df["No_Points_Dim_2"] = choose_column_in_matrix(list(nos_points), 2)


    feature_df["Largest_Persistence_Dim_0"] = persistences[0]
    feature_df["Largest_Persistence_Dim_1"] = persistences[1]
    feature_df["Largest_Persistence_Dim_2"] = persistences[2]

    # Label
    feature_df["Label"] = label

    return feature_df

In [25]:
train_dataframes = {}
test_dataframes= {}

for label in label_list:
    train_dataframes[label] = create_feature_df(train_feautures[label][0], train_feautures[label][1], train_feautures[label][2], \
                                                train_largest_persistences[label], label)
    
    test_dataframes[label] = create_feature_df(test_feautures[label][0], test_feautures[label][1], test_feautures[label][2], \
                                                test_largest_persistences[label], label)

In [29]:
# Concatenate and save features of training persistence diagrams
train_feature_df = pd.concat([train_dataframes[0], train_dataframes[1], train_dataframes[2], train_dataframes[3], train_dataframes[4]], ignore_index=True)
train_feature_df.to_csv("Features/Train_Topological_Summary_Statistics.csv")

# Concatenate and save features of training persistence diagrams
test_feature_df = pd.concat([test_dataframes[0], test_dataframes[1], test_dataframes[2], test_dataframes[3], test_dataframes[4]], ignore_index=True)
test_feature_df.to_csv("Features/Test_Topological_Summary_Statistics.csv")